In [1]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
import gzip
import random
import scipy
import tensorflow as tf
from collections import defaultdict
from implicit import bpr

def readGz(path):
    for l in gzip.open(path, 'rt',encoding='utf8'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [2]:
allRatings = []

userRatings = defaultdict(list)
recipeRatings = defaultdict(list)

for user,recipe,d in readCSV("C:/Users/brand/Desktop/trainInteractions.csv.gz"):
    r = int(d['rating'])
    allRatings.append(r)
    userRatings[user].append(r)
    recipeRatings[recipe].append(r)

Mu = sum(allRatings) / len(allRatings)

In [3]:
userAverage = {}
itemAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
    
for i in recipeRatings:
    itemAverage[i] = sum(recipeRatings[i])/len(recipeRatings[i])


userIDs = {}
recipeIDs = {}
interactions = []

for user,recipe,d in readCSV("C:/Users/brand/Desktop/trainInteractions.csv.gz"):
    u = user
    i = recipe
    r = int(d['rating'])
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in recipeIDs: recipeIDs[i] = len(recipeIDs)
    interactions.append((u,i,r))

interTrain = interactions[0:400000]
interVal = interactions[400000:]



In [4]:
import pandas as pd
    
def testvalsplit(path):
    
    f = gzip.open(path, 'rt')
    data = pd.read_csv(f,dtype=str)    
    train = data[0:400000]
    val = data[400000:]
    
    
    
    return data,train,val


d,t,v = testvalsplit("C:/Users/brand/Desktop/trainInteractions.csv.gz")

print(len(itemAverage))
print(len(userAverage))




151462
13533


In [5]:
v['cooked'] = 1
print(v.iloc[0,:])


recipelst = []
recipePerUser = defaultdict(set)


for index,row in d.iterrows():
    use = row['user_id']
    rec = row['recipe_id']
    recipelst.append(rec)
    recipePerUser[use].add(rec)
    
recipelst = set(recipelst)
recipelst = list(recipelst)


C:\Users\brand\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


user_id        90764166
recipe_id      01768679
date         2011-09-10
rating                5
cooked                1
Name: 400000, dtype: object


In [25]:
import numpy as np
import random

recipeset = set(recipelst)


lst = []


for index, row in v.iterrows():
    use = row['user_id']
    rec = row['recipe_id']
    date = row['date']
    rat = 0
    indx = np.random.randint(0,len(recipelst))
    new_recipe = recipelst[indx]
    lst.append([use,new_recipe,date,rat,0])

In [26]:
columns = ['user_id','recipe_id','date','rating','cooked']
index = np.arange(500000,600000)

df = pd.DataFrame(lst,index,columns)

val_set = pd.concat([v,df])

print(len(val_set))
print(val_set.head())


200000
         user_id recipe_id        date rating  cooked
400000  90764166  01768679  2011-09-10      5       1
400001  68112239  24923981  2007-03-30      5       1
400002  32173358  57597698  2009-01-16      4       1
400003  30893740  16266088  2010-06-12      5       1
400004  69780905  62953151  2004-02-10      5       1


In [7]:
recipeCount = defaultdict(int)
totalCooked = 0

for index,row in t.iterrows():
    recipeCount[row['recipe_id']] += 1
    totalCooked += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()


'''
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalCooked/2: 
        break

#correct = 0


for index,row in val_set.iterrows():
    if row['recipe_id'] in return1:
        correct += (row['cooked'] != 0)
    
    else:
        correct += (row['cooked'] == 0)

print('Validation set accuracy:' + str(correct/len(val_set)))
'''

"\nreturn1 = set()\ncount = 0\nfor ic, i in mostPopular:\n    count += ic\n    return1.add(i)\n    if count > totalCooked/2: \n        break\n\n#correct = 0\n\n\nfor index,row in val_set.iterrows():\n    if row['recipe_id'] in return1:\n        correct += (row['cooked'] != 0)\n    \n    else:\n        correct += (row['cooked'] == 0)\n\nprint('Validation set accuracy:' + str(correct/len(val_set)))\n"

In [8]:
Xsparse = scipy.sparse.lil_matrix((len(recipeRatings),len(userRatings)))

for index,row in t.iterrows():
    u,i = row['user_id'],row['recipe_id']
    Xsparse[recipeIDs[i],userIDs[u]] = 1
    
Xsparse = scipy.sparse.csr_matrix(Xsparse.T)

In [9]:
model = bpr.BayesianPersonalizedRanking(factors=20)

model.fit(Xsparse)

100%|██████████| 100/100 [00:02<00:00, 44.94it/s, correct=90.80%, skipped=0.82%]


In [13]:


t['cooked'] = 1

print(t.head())

"""All items in training set are cooked. Must incorporate noncooked items into training set for logistic regression"""

tlst = []

for index, row in t.iterrows():
    use = row['user_id']
    rec = row['recipe_id']
    date = row['date']
    rat = 0
    
    indx = np.random.randint(0,len(recipelst))
    new_recipe = recipelst[indx]
    tlst.append([use,new_recipe,date,rat,0])
    
columns = ['user_id','recipe_id','date','rating','cooked']
index = np.arange(400000,800000)

df2 = pd.DataFrame(tlst,index,columns)

train = pd.concat([t,df2])

print(len(train))
print(train.head())
print(train.tail())


C:\Users\brand\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


    user_id recipe_id        date rating  cooked
0  88348277  03969194  2004-12-23      5       1
1  86699739  27096427  2002-01-12      4       1
2  03425965  44197323  2012-10-03      5       1
3  73973193  24971400  2008-04-09      5       1
4  15215209  60170202  2010-10-07      5       1
800000
    user_id recipe_id        date rating  cooked
0  88348277  03969194  2004-12-23      5       1
1  86699739  27096427  2002-01-12      4       1
2  03425965  44197323  2012-10-03      5       1
3  73973193  24971400  2008-04-09      5       1
4  15215209  60170202  2010-10-07      5       1
         user_id recipe_id        date rating  cooked
799995  54496210  41675729  2008-08-23      0       0
799996  43613845  05905421  2006-06-06      0       0
799997  20028793  25325191  2009-03-08      0       0
799998  97985933  68797434  2004-06-24      0       0
799999  11959868  74909928  2007-06-12      0       0


In [14]:
return1 = set()
return2 = set()
return3 = set()
count = 0
for ic, i in mostPopular:
    if count < totalCooked/4:
        count += ic
        return1.add(i)
    
    elif count >= totalCooked/4 and count < totalCooked/2:
        count += ic
        return2.add(i)
    
    elif count >= totalCooked/2 and count < 3*totalCooked/4:
        count +=ic
        return3.add(i)
        
    
    else:
        break
        

print(len(return1) + len(return2) + len(return3))
print(return1.intersection(return2))
print(return2.intersection(return3))
print(return3.intersection(return1))
    
   

50010
set()
set()
set()


In [15]:
def Jaccard(s1,s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    
    return numer/denom

usersPerRecipe = defaultdict(set)
recipesPerUser = defaultdict(set)

for index, row in t.iterrows():
    user,recipe = row['user_id'],row['recipe_id']
    usersPerRecipe[recipe].add(user)
    recipesPerUser[user].add(recipe)



            


In [16]:
def jacfinder(user,item):
    
    jacs = []
        
    recipeset = list(recipesPerUser[user])
        
    for i in recipeset:
        if i == item:
            continue
            
        sim = Jaccard(usersPerRecipe[i],usersPerRecipe[item])
        jacs.append(sim)
    
    if len(jacs) > 0:
        return max(jacs)
    
    return 0
            

In [17]:
features = []

itemFactors = model.item_factors
userFactors = model.user_factors


'''items and users are switched because model was built on transpose of matrix'''


for index,row in train.iterrows():
    lst = []
    use = row['user_id']
    rec = row['recipe_id']
    cook = row['cooked']
    jac = jacfinder(use,rec)
    pop = [0,0,0]
    item_factors_index = userIDs[use]
    user_factors_index = recipeIDs[rec]
    user_factors = itemFactors[item_factors_index]
    item_factors = userFactors[user_factors_index]
    
    for i in user_factors:
        lst.append(i)
        
    for j in item_factors:
        lst.append(j)
        
    if rec in return1:
        pop[0] = 1
    
    if rec in return2:
        pop[1] = 1
        
    if rec in return3:
        pop[2] = 1
    
    lst.append(jac)
    
    for k in pop:
        lst.append(k)
        
    lst.append(cook)
    
    features.append(lst)

print(features[0])





[-1.3478975, 0.30834872, -0.38887686, 0.25709724, 1.3696382, -0.82923603, 1.0554105, -0.68341345, -0.5220155, -0.5220077, 1.257014, -1.218342, -0.12035619, 0.50496954, -0.19915546, -0.615369, -0.010041287, 0.42375857, 1.5902828, 0.43235794, -0.45055562, -0.31654572, 0.34982523, -0.26176143, 0.006704752, 0.25558725, -0.021249041, 0.26165432, -0.04160688, 0.2106823, 0.024320116, 0.2701392, -0.18119806, 0.008939141, 0.13209063, 0.06041354, 0.07439145, 0.09460471, -0.05647614, 0.30153677, -0.3362417, 1.0, 0.2, 0, 1, 0, 1]


In [69]:
print(len(features[0]))
print(train.head())


47
    user_id recipe_id        date rating  cooked
0  88348277  03969194  2004-12-23      5       1
1  86699739  27096427  2002-01-12      4       1
2  03425965  44197323  2012-10-03      5       1
3  73973193  24971400  2008-04-09      5       1
4  15215209  60170202  2010-10-07      5       1


In [19]:
columns = np.arange(0,47)
index = np.arange(0,800000)

dftrain = pd.DataFrame(features,index,columns)
#print(dftrain.head())


xtrain = dftrain[np.arange(0,46)]
ytrain = dftrain[[46]]
ytrain = np.array(ytrain)
ytrain = ytrain.flatten()


In [20]:
features2 = []

itemFactors = model.item_factors
userFactors = model.user_factors


'''items and users are switched because model was built on transpose of matrix'''


for index,row in val_set.iterrows():
    lst = []
    use = row['user_id']
    rec = row['recipe_id']
    cook = row['cooked']
    jac = jacfinder(use,rec)
    pop = [0,0,0]
    item_factors_index = userIDs[use]
    user_factors_index = recipeIDs[rec]
    user_factors = itemFactors[item_factors_index]
    item_factors = userFactors[user_factors_index]
    
    for i in user_factors:
        lst.append(i)
        
    for j in item_factors:
        lst.append(j)
        
    if rec in return1:
        pop[0] = 1
    
    if rec in return2:
        pop[1] = 1
        
    if rec in return3:
        pop[2] = 1
    
    lst.append(jac)
    
    for k in pop:
        lst.append(k)
        
    lst.append(cook)
    
    features2.append(lst)

print(features2[0])


[-0.07175326, -0.8379519, 0.87732226, 0.04911443, -0.3386144, 0.7689721, 0.4267238, -0.40463054, 0.09862076, 0.00972563, -2.1377075, 0.74528915, 0.24726506, 0.7377541, 0.716959, -1.0289389, 1.7711493, -0.103238985, 0.63920176, 0.37610194, -0.37779573, 0.018907947, 0.45346135, 0.5770398, 0.3886331, -0.043311987, -0.30446708, -0.053528037, -0.11054357, 0.36426923, -0.0007320005, -0.10639218, 0.13408092, -0.09890947, -0.21855304, 0.045118924, 0.2084963, -0.27869982, -0.012766888, -0.3493072, -0.115708984, 1.0, 0.08333333333333333, 0, 1, 0, 1]


In [31]:
from sklearn.linear_model import LogisticRegression


columns = np.arange(0,47)
index2 = np.arange(0,200000)
dfval = pd.DataFrame(features2,index2,columns)

xval = dfval[np.arange(0,46)]
yval = dfval[[46]] 
yval = np.array(yval)
yval = yval.flatten()





log = LogisticRegression(class_weight='balanced', C=0.0001).fit(xtrain,ytrain)


preds = log.predict(xval)

In [77]:
error = [(p == l) for p,l in zip(preds,yval)]

print(sum(error)/len(error))
print(error[0:10])
print(sum(preds))
print(len(preds))


0.66228
[True, False, False, False, True, False, False, True, False, False]
53662
200000


In [80]:
predictions = open("C:/Users/brand/Desktop/predictions_Made.txt", 'w')
for l in open("C:/Users/brand/Desktop/stub_Made.txt"):
    if l.startswith("user_id"):
    #header
        predictions.write(l)
        continue
    
    u, i = l.strip().split('-')
    
    pred = 0
    user_recipes = list(recipesPerUser[u])
        
    for j in user_recipes:
        if j == i:
            continue
        sim = Jaccard(usersPerRecipe[j],usersPerRecipe[i])
            
        if sim > 0.008:
            pred = 1
        
    if (i in return1 or i in return2) and pred ==1:
        predictions.write(u + '-' + i + ",1\n")
        continue
            
        
    else:
        predictions.write(u + '-' + i + ",0\n")
        continue
    
   

predictions.close()

In [17]:
items = list(recipeIDs.keys())


class BPRbatch(tf.keras.Model):
    def __init__(self, K, lamb):
        super(BPRbatch, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(recipeIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(recipeIDs),K],stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb

    # Prediction for a single instance
    def predict(self, u, i):
        p = self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +\
                            tf.nn.l2_loss(self.gammaU) +\
                            tf.nn.l2_loss(self.gammaI))
    
    def score(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        x_ui = beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return x_ui

    def call(self, sampleU, sampleI, sampleJ):
        x_ui = self.score(sampleU, sampleI)
        x_uj = self.score(sampleU, sampleJ)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_ui - x_uj)))        

In [18]:
optimizer = tf.keras.optimizers.Adam(0.01)

modelBPR = BPRbatch(20, 0.000015)



In [19]:
def trainingStepBPR(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ = [], [], []
        for _ in range(Nsamples):
            u,i,_ = random.choice(interactions) # positive sample
            j = random.choice(items) # negative sample
            while j in recipesPerUser[u]:
                j = random.choice(items)
            sampleU.append(userIDs[u])
            sampleI.append(recipeIDs[i])
            sampleJ.append(recipeIDs[j])

        loss = model(sampleU,sampleI,sampleJ)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [50]:
for i in range(100):
    obj = trainingStepBPR(modelBPR, interTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.56568336
iteration 20, objective = 0.5634785
iteration 30, objective = 0.5639291
iteration 40, objective = 0.5639781
iteration 50, objective = 0.564836
iteration 60, objective = 0.56491697
iteration 70, objective = 0.566018
iteration 80, objective = 0.5615854
iteration 90, objective = 0.561391
iteration 100, objective = 0.56184155


In [22]:
import math
u,i,_ = interVal[0]

modelBPR.predict(userIDs[u],recipeIDs[i]).numpy()
correct = 0

pos = []
count = 0
for i in range(len(interTrain)):
    user,recipe,_ = interTrain[i]
    score = modelBPR.predict(userIDs[user],recipeIDs[recipe]).numpy()
    pos.append(score)
    count +=1 
    
    if count == 100000 or count == 250000 or count == 450000:
        print('hi')


    
    
print(min(pos))
print(max(pos))
  






hi
hi
-2.0893147
10.368451


In [28]:
t_avg = sum(pos)/len(pos)
print(t_avg)

pos.sort(reverse=True)

values = []
liverpool = [1.2,1.25]

for i in liverpool:
    correct = 0
    for index,row in val_set.iterrows():
    
        use = row['user_id']
        rec = row['recipe_id']
        guess = 0
        pred = modelBPR.predict(userIDs[use],recipeIDs[rec]).numpy()
        if pred > pos[round(len(pos)/i)]:
            guess = 1
    
        if guess == row['cooked']:
            correct += 1
        
        
        
    print(correct/len(val_set))
    values.append(correct/len(val_set))
    correct = 0

print(values)


0.8213277312667646
0.675895
0.68108
[0.675895, 0.68108]


In [51]:
return1 = set()

count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalCooked/1.7:
        break

In [31]:
correct = 0
for index,row in val_set.iterrows():
    
    jthresh = 0
    popthresh = 0
    bprthresh = 0
    guess = 0
    
    user,recipe = row['user_id'],row['recipe_id']
    
    recipeset = list(recipesPerUser[user])
    
    for i in recipeset:
        if i == recipe:
            continue
        
        sim = Jaccard(usersPerRecipe[i],usersPerRecipe[recipe])
        
        if sim > 0.0086:
            jthresh = 1
            break
     
        
    if recipe in return1:
        popthresh = 1
        
    pred = modelBPR.predict(userIDs[user],recipeIDs[recipe]).numpy()
    
    if pred > pos[round(len(pos)/1.27)]:
        bprthresh = 1
    
    
    if jthresh == 1 and bprthresh == 1 and popthresh == 1:
        guess = 1
    
    if guess == row['cooked']:
        correct += 1
    
print(correct/len(val_set))
        
    
        
    
        

0.66448


In [52]:
import random
count = 0

predictions = open("C:/Users/brand/Desktop/predictions_Made.txt", 'w')
for l in open("C:/Users/brand/Desktop/stub_Made.txt"):
    if l.startswith("user_id"):
    #header
        predictions.write(l)
        continue
    
    u, i = l.strip().split('-')
    
    if u not in userIDs.keys() or i not in recipeIDs.keys():
        pred = 0
        user_recipes = list(recipesPerUser[u])
        
        for j in user_recipes:
            if j == i:
                continue
            sim = Jaccard(usersPerRecipe[j],usersPerRecipe[i])
            
            if sim > 0.0086:
                pred = 1
        
        if i in return1 and pred ==1:
            predictions.write(u + '-' + i + ",1\n")
            continue
        
        else:
            predictions.write(u + '-' + i + ",0\n")
            continue
    
    #jthresh = 0
    #popthresh = 0
    bprthresh = 0
    guess = 0
    
    #recipeset = list(recipesPerUser[u])
    
    #for j in recipeset:
     #   if j == i:
      #      continue
        
       # sim = Jaccard(usersPerRecipe[j],usersPerRecipe[i])
        
        #if sim > 0.0086:
         #   jthresh = 1
          #  break
        
        
    #if i in return1:
     #   popthresh = 1
        
    pred = modelBPR.predict(userIDs[u],recipeIDs[i]).numpy()
    
    if pred > pos[round(len(pos)/1.33)] or i in return1:
        bprthresh = 1
    
    if bprthresh == 1:
        guess = 1

    
    predictions.write(u+ '-' + i + "," + str(guess) + "\n")
        

predictions.close()
